<h1><p style="text-align: center;"> SQL. Анализ базы данных сервиса для чтения книг по подписке </p></h1>
<h3> Содержание </span></h3>

<div class="toc">
    <ol class="toc-item">
        <li> Исследование таблиц </li>
        <li> SQL-запросы для решения заданий и вывод результатов </li>
        <ul>
            <li> подсчёт количества книг, вышедших после 1 января 2000 года </li>
            <li> подсчёт количества обзоров и средней оценки для каждой книги </li>
            <li> определение издательства, которое выпустило наибольшее число книг толще 50 страниц ( исключаем из анализа брошюры) </li>
            <li> определение автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками) </li>
            <li> подсчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок </li>
        </ul> 
        <li> Выводы </li>
    </ol>
</div>

## 1. Исследование таблиц 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': '', # имя пользователя
             'pwd': '', # пароль
             'host': '',
             'port': , # порт подключения
             'db': ''} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выводим первые строки таблицы с информацией о книгах

In [2]:
query = '''
           SELECT *
           FROM books
           LIMIT 5
        '''
books = pd.io.sql.read_sql(query, con=engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выводим первые строки таблицы с информацией об авторах

In [3]:
query = '''
           SELECT *
           FROM authors
           LIMIT 5
        '''
authors = pd.io.sql.read_sql(query, con=engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выводим первые строки таблицы с информацией о издательствах

In [4]:
query = '''
           SELECT *
           FROM publishers
           LIMIT 5
        '''
publishers = pd.io.sql.read_sql(query, con=engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выводим первые строки таблицы с информацией о рейтингах

In [5]:
query = '''
           SELECT *
           FROM ratings
           LIMIT 5
        '''
ratings = pd.io.sql.read_sql(query, con=engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выводим первые строки таблицы с информацией об обзорах

In [6]:
query = '''
           SELECT *
           FROM reviews
           LIMIT 5
        '''
reviews = pd.io.sql.read_sql(query, con=engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Первые строки каждой из таблиц мы посмотрели, информация о типах данных в каждой из колонок каждой из таблиц нам известна из задания, типы данных для дальнейшей работы коррекции не требуют.

## 2.  SQL-запросы для решения заданий и вывод результатов

#### - подсчёт количества книг, вышедших после 1 января 2000 года

В данном запросе нам понадобятся данные только из таблицы с информацией о книгах, ограничим их датой публиции, указанной в задании

In [7]:
query = '''
            SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01';
        '''
books_quantity = pd.io.sql.read_sql(query, con=engine)
books_quantity

,count
0,819


После 1 января 2000 года было выпущено 819 книг, в целом, есть ощущение, что это не так много, но лучше проверить, каким периодом датируется окончание нашей выборки.Выведем информацию о крайних датах изданий нашей таблицы.

In [8]:
query = '''
            SELECT MIN(publication_date),
                   MAX(publication_date)
            FROM books;
        '''
books_quantity = pd.io.sql.read_sql(query, con=engine)
books_quantity

,min,max
0,1952-12-01,2020-03-31


Итак, мы обладаем данными до конца марта 2020, значит, за период в чуть больше 20 лет было выпущено всего 819 книг (чуть больше 40 книг в год), кажется, что это совсем мало, но давайте проверим общее количество книг в нашей выборке.

In [9]:
query = '''
            SELECT COUNT(book_id)
            FROM books;
        '''
books_quantity = pd.io.sql.read_sql(query, con=engine)
books_quantity

,count
0,1000


Всего в нашей выборке 1000 книг, таким образом, выходит, что более 80% книг были выпущены в последние 20 лет, а оставшиеся чуть менее 20% были выпущены в период с 1952 по 1999 года включительно.

#### - подсчёт количества обзоров и средней оценки для каждой книги

В данном запросе мы оформляем два подзапроса: в первом связываем между собой таблицы с информацией о книгах и обзорах, считаем количество обзоров для каждой книги, во втором - связываем таблицы с информацией о книгах  и оценках, выводим среднюю оценку для каждой книги, из этих двух подзапросов выводим информацию о книге, количестве обзоров и средней оценке, сортируем список по убыванию количества обзоров.

In [10]:
query = '''
            WITH 
            rq AS 
                    (SELECT b.book_id,
                            b.title,
                            COUNT(rev.review_id) AS reviews_quantity
                     FROM books AS b
                     FULL OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
                     GROUP BY b.book_id),
            
            sc AS
                    (SELECT b.book_id,
                            b.title,
                            AVG(rat.rating) AS avg_rating_score
                     FROM books AS b
                     FULL OUTER JOIN ratings AS rat ON b.book_id = rat.book_id
                     GROUP BY b.book_id)
            SELECT rq.book_id,
                   rq.title,
                   rq.reviews_quantity,
                   sc.avg_rating_score
            FROM rq JOIN sc ON rq.book_id = sc.book_id
            ORDER BY reviews_quantity DESC;
        '''
rev_n_rat = pd.io.sql.read_sql(query, con=engine)
rev_n_rat

,book_id,title,reviews_quantity,avg_rating_score
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Видим, что наибольшее количество обзоров получила книга Twilight (Сумерки) - 7 обзоров, Возможно, наибольшее количество обзоров именно у этой книги ввиду популярности одноименного фильма среди девочек-подростков, однако, как мы видим, что средняя оценка у этой книги не столь высока, попробуем отсортировать список по убыванию средней оценки, чтобы узнать лучшую книги по мнению пользователей.

In [11]:
query = '''
            WITH 
            rq AS 
                    (SELECT b.book_id,
                            b.title,
                            COUNT(rev.review_id) AS reviews_quantity
                     FROM books AS b
                     FULL OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
                     GROUP BY b.book_id),
            
            sc AS
                    (SELECT b.book_id,
                            b.title,
                            AVG(rat.rating) AS avg_rating_score
                     FROM books AS b
                     FULL OUTER JOIN ratings AS rat ON b.book_id = rat.book_id
                     GROUP BY b.book_id)
            SELECT rq.book_id,
                   rq.title,
                   rq.reviews_quantity,
                   sc.avg_rating_score
            FROM rq JOIN sc ON rq.book_id = sc.book_id
            ORDER BY avg_rating_score DESC
            LIMIT 50;
        '''
rev_n_rat = pd.io.sql.read_sql(query, con=engine)
rev_n_rat

,book_id,title,reviews_quantity,avg_rating_score
0,55,A Woman of Substance (Emma Harte Saga #1),2,5.000000
1,182,Dead Souls,2,5.000000
2,901,The Walking Dead Book One (The Walking Dead #...,2,5.000000
3,62,Alas Babylon,2,5.000000
4,967,Welcome to Temptation (Dempseys #1),2,5.000000
5,224,Evening Class,2,5.000000
6,610,Tai-Pan (Asian Saga #2),2,5.000000
7,699,The Demon-Haunted World: Science as a Candle i...,2,5.000000
8,993,Women,2,5.000000
9,76,Angels Fall,2,5.000000


Оказывается, у 43 книг наивысшая средняя оценка, правда, обзоров на эти книги совсем мало (в основном по 2 обзора), поэтому эти книги нравятся тем, кто их прочёл, но прочли их немногие, видимо, либо мало кто захотел писать на них обзор после прочтения.

#### - определение издательства, которое выпустило наибольшее число книг толще 50 страниц ( исключаем из анализа брошюры)

В данном случае мы связываем между собой таблицы с информацией о книгах и об издательствах, ограничиваем выборку количеством страниц в книге, сортируем список по убыванию количества книг и выводим информацию об издательстве, которое выпустило самое большое количество книг (не брошюр).

In [12]:
query = '''
            SELECT p.publisher_id,
                   p.publisher,
                   COUNT(b.book_id) AS books_quantity    
            FROM books AS b
            LEFT JOIN publishers AS p on b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY books_quantity DESC
            LIMIT 1;
        '''
largest_publisher = pd.io.sql.read_sql(query, con=engine)
largest_publisher

,publisher_id,publisher,books_quantity
0,212,Penguin Books,42


Итак, выяснили, что таким издательством является Penguin Books, выпустившее 42 книги.
Как известно, когда-то это издательство одним из первых стало выпускать серьёзные книги в несерьёзных мягких обложках. Однако, мы видим, что 42 книги это не так много, видимо, в целом в нашей выборке книги распределились относительно равномерно по издательствам или в таблицах есть дубликаты наименований издательств, поэтому наш результат может оказаться не вполне корректным.
Поэтому попробуем вывести список из 30 наиболее печатающих издательств.

In [13]:
query = '''
            SELECT p.publisher_id,
                   p.publisher,
                   COUNT(b.book_id) AS books_quantity    
            FROM books AS b
            LEFT JOIN publishers AS p on b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY books_quantity DESC
            LIMIT 30;
        '''
largest_publisher = pd.io.sql.read_sql(query, con=engine)
largest_publisher

,publisher_id,publisher,books_quantity
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,284,St. Martin's Press,14
8,46,Berkley Books,14
9,83,Delta,13


Как и ожидали, видим уже, как минимум, три варианта написания пингвинов в выборке из 30 издательств, очевидно, данные в таблице с издательствами требуют определённого "причёсывания", чтобы можно было делать выводы о достоверности полученного результата.

#### - определение автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)
    
В данном случае, объединяем таблицы с информацией о рейтингах, книгах и авторах, группируем данные по авторам, выводим среднюю оценку книг автора, сортируем список по убыванию оценки, выводим первую строку.

In [14]:
query = '''
            SELECT AVG(rat.rating) AS avg_score,
                   a.author
            FROM ratings AS rat
            RIGHT JOIN books AS b ON rat.book_id = b.book_id
            LEFT JOIN authors AS a ON b.author_id = a.author_id
            GROUP BY a.author
            HAVING COUNT(rat.rating_id) > 50
            ORDER BY avg_score DESC
            LIMIT 1;
        '''
highest_score = pd.io.sql.read_sql(query, con=engine)
highest_score

,avg_score,author
0,4.288462,J.K. Rowling/Mary GrandPré


Ожидаемо, в топе наиболее высокооцениваемых авторов у нас автор Гарри Поттера - J.K. Rowling, правда, тут автор идёт в паре с иллюстратором Mary GrandPré, и мы значем, что серия книг про Гарри Поттера в компании именно с этим иллюстратором имела наибольший успех, так что тут, возможно, заслуга не только автора книги.
Посмотрим на первую десятку наиболее высокооцениваемых авторов.

In [15]:
query = '''
            SELECT AVG(rat.rating) AS avg_score,
                   a.author
            FROM ratings AS rat
            RIGHT JOIN books AS b ON rat.book_id = b.book_id
            LEFT JOIN authors AS a ON b.author_id = a.author_id
            GROUP BY a.author
            HAVING COUNT(rat.rating_id) > 50
            ORDER BY avg_score DESC
            LIMIT 10;
        '''
highest_score = pd.io.sql.read_sql(query, con=engine)
highest_score

,avg_score,author
0,4.288462,J.K. Rowling/Mary GrandPré
1,4.283019,Agatha Christie
2,4.264151,Markus Zusak/Cao Xuân Việt Khương
3,4.240964,J.R.R. Tolkien
4,4.209677,Roald Dahl/Quentin Blake
5,4.203704,Louisa May Alcott
6,4.130952,Rick Riordan
7,4.107143,Arthur Golden
8,4.009434,Stephen King
9,3.971429,John Grisham


Видим, что у первых двух авторов средняя оценка очень близка, а авторы их все1 первой десятки действительно авторы мировых бестселлеров, так что, полагаю, по данным в таблице с авторами "сюрпризов" в виде дубликатов, скорее всего, нет.

#### - подсчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок

Для выполнения такого расчёта нам необходимо сформировать подзапрос, в котором мы подсчитаем количество обзоров, которые оставил каждый пользователь, причём пользователей будем брать из таблицы с рейтингами, ограничив выборку лишь теми, кто поставил более 50 оценок. Далее из получившего подзапроса нам остаётся лишь подсчитать среднее количество обзоров от выбранных пользователей.

In [16]:
query = '''
            WITH 
            users_reviews AS 
                    (SELECT DISTINCT(username) AS username,
                            COUNT(review_id) AS reviews_quantity
                     FROM reviews 
                     WHERE username in (SELECT DISTINCT(username)
                                        FROM ratings 
                                        GROUP BY username
                                        HAVING COUNT(book_id) > 50)
                     GROUP BY username)
            SELECT AVG(reviews_quantity)
            FROM users_reviews;
        '''
avg_reviews = pd.io.sql.read_sql(query, con=engine)
avg_reviews

,avg
0,24.333333


Выходит, что среднее количество обзоров на одного пользователя, который любит ставить книгам оценки,  - 24 обзора, довольно много, похоже, что такие пользователи уже близки к уровню литературных критиков.
Но попробуем дополнительно посмотреть на выборку из пользователей, которые любят ставить оценки, выясним, сколько их в целом, возможно, среди них есть те, кто пишет очень много обзоров, а кто-то пишет совсем мало.

In [17]:
query = '''
            SELECT DISTINCT(username) AS username,
                   COUNT(review_id) AS reviews_quantity
            FROM reviews 
            WHERE username in (SELECT DISTINCT(username)
                               FROM ratings 
                               GROUP BY username
                               HAVING COUNT(book_id) > 50)
            GROUP BY username
            ORDER BY reviews_quantity DESC;
        '''
avg_reviews = pd.io.sql.read_sql(query, con=engine)
avg_reviews

,username,reviews_quantity
0,sfitzgerald,28
1,martinadam,27
2,richard89,26
3,jennifermiller,25
4,paul88,22
5,xdavis,18


Видим, что у нас всего 5 литературных критиков, которые действительно пишут много обзоров (от 18 до 28), так что среднее значение выглядит максимально корректным.

## 3. Выводы

анализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.Итак, мы проанализировали имеющуюся базу данных с информацией о книгах, издательствах, авторах, пользовательских обзорах и оценках книг. 
В ходе исследования мы выполнили следующие шаги:
- выяснили количество книг, вышедших за прошедшее время (после 1 января 2000 года) - всего 819 таких книг и учитывая, что в нашей выборке есть данные с начала 1952 до конца марта 2020 года, а книг в выборке всего 1000, выходит в последние 20 лет были изданы более 80% книг из выборки, это может говорить как о том, что либо издавать книги стало дешевле/проще, так и о том, что спрос на них в последние годы вырос.
- оценили обозреваемость и узнали среднюю оценку для каждой книги из имеющейся базы данных - оказалось что больше всего обзоров у книги "Сумерки", возможно,из-за популярности одноимённого фильма, при этом рейтинг книги не очень высокий, а наиболее высокие рейтинги у книг, на которые мало обзоров, выходит, либо об этих книгах знают немногие читатели, в связи с чем не могут их прочесть, либо это книги специфические и рассчитаны на "своего" читателя, который их и находит, либо пользователи просто не делают обзоры на них по каким-то своим причинам.
- выделили издательство, которое выпустило наибольшее число книг - это издательство британское издательство Penguin, однако тут возможны варианты, так как мы обнаружили некоторые неявные дубликаты в данных по наименованиям издательств (есть написания части имени издательства, написание серии книг, которые выпускает издательство, и т.п.), имеет смысл перепроверить информацию об издательстве, которое выпустило наибольшее число книг, после обработки этих дубликатов. 
- нашли автора с наивысшей (по мнению пользователей) средней оценкой книг среди наиболее оцениваемых книг - это автор Гарри Поттера в компании с наиболее известным иллюстратором книг о Гарри Поттере (J.K. Rowling/Mary GrandPré), также в выборке с наиболее высокой средней оценкой есть авторы современных и уже не очень современных (Агата Кристи) бестселлеров.
- узнали, сколько в среднем обзоров книг делают пользователи, оценивающие книги максимально часто - оказалось что обозревателей, которые чаще других оценивают книги, всего 5, в среднем они делают по 24 обзора (от 18 до 28).

Что касается цели исследования - сформулировать ценностное предложение для нового продукта в приложении для чтения книг по подписке, то, можно предложить внедрить в приложении следующие опции:
- диверсификация предложений по книгам для разных категорий читателей (по возрасту и  интересам)
- возможность прочитать первую главу книги бесплатно (чтобы пользователь мог оценить, насколько ему интересно продолжать чтение и затем подписаться на неё)
- информация о наиболее высокооценённых книгах на главной странице приложения, чтобы пользователи сразу могли её увидеть
- функционал по подбору похожей (по жанру, стилистике, автору) книги, после прочтения предыдущей, особенно, если пользователь поставил такой книге высокую оценку
- предложение оценить книгу сразу после прочтения (лучше с возможностью не оставлять обязательный текст или проходить опрос, не все пользователи хотят тратить дополнительное время на это, некоторым проще просто поставить нужное количество звёздочек, чем  объяснять, что конкретно понравилось, а что нет).